In [3]:
import numpy as np
import os

from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard
from keras.layers import Input, Dense, Flatten, Convolution2D, BatchNormalization, Activation, Add
from keras.models import Model
from keras.utils import to_categorical
from keras import backend as K
from pommerman.constants import BOARD_SIZE
from tqdm import tqdm_notebook as tqdm
from sklearn.utils.class_weight import compute_class_weight

In [4]:
batch_size = 2048
epochs = 150
early_stopping = 5
action_space = 6

log_path = './supervised_learning/logs/1cnn/'
model_path = './supervised_learning/model/1cnn/model.h4'

train_data_path    = './dataset/'
train_data_labels  = os.path.join(train_data_path, 'labels.npy')
train_data_reward  = os.path.join(train_data_path, 'reward.npy')
train_data_obs_map = os.path.join(train_data_path, 'obs_map.npy')

if not os.path.isdir(train_data_path):
    os.makedirs(train_data_path)

In [5]:
class Trainer:
    def __init__(self, actions, save_path, log_path, save_best_only=True, seed=0):
        K.clear_session()
        self.log_path = log_path
        self.save_path = save_path
        self.actions = actions
        self.save_best_only = save_best_only
        
        # Create model
        self.model = self.create_model(actions)
        # Load model if exists
        if not os.path.isdir(os.path.dirname(save_path)):
            os.makedirs(os.path.dirname(save_path))            
        if os.path.isfile(self.save_path):
            try:
                print("Trying to load model")
                self.model.load_weights(self.save_path)
                print("Model was loaded successful")
            except:
                print("Model load failed")
        
    def get_res_block(self, input):
        # Res block 1        
        x = Convolution2D(256, 3, padding='same')(input)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Convolution2D(256, 3, padding='same')(x)
        x = BatchNormalization()(x)
        x = Add()([input, x])
        x = Activation('relu')(x)
        return x
        
    def create_model(self, actions, input_shape=(11, 11, 18,)):
        inp = Input(input_shape)
        # 5 Convs -> receptive field 11
        x = Convolution2D(256, 3, padding='valid')(inp)
        x = BatchNormalization()(x)
        x = Activation('elu')(x)
        x = Flatten()(x)
        # Output
        probs  = Dense(actions, activation='softmax', name='actions')(x)
        reward = Dense(1, activation='tanh', name='reward')(x)
        
        model = Model(inputs = inp, outputs=[probs, reward])
        model.compile(optimizer='adam', loss=['categorical_crossentropy', 'mae'], metrics=['accuracy'])
        return model

    def train(self, obs, actions, rewards, batch_size=16384, epochs=100,
              early_stopping = 10, class_weight=None, initial_epoch=0):
        
        early_stopping = EarlyStopping(monitor='loss', patience=early_stopping)
        checkpoint     = ModelCheckpoint(self.save_path, monitor='loss', save_best_only=self.save_best_only)
        reduce_lr      = ReduceLROnPlateau(monitor='loss', patience=2, factor=0.1)
        logger         = CSVLogger(self.log_path + 'log.csv', append=True)
        tensorboard    = TensorBoard(self.log_path, batch_size=batch_size)
        
        history = self.model.fit(x=obs, y=[actions, rewards], batch_size=batch_size, epochs=epochs, verbose=1,
                       callbacks=[early_stopping, checkpoint, reduce_lr, logger, tensorboard],
                       validation_split=0.15, shuffle=True, class_weight=class_weight, initial_epoch=initial_epoch)

In [6]:
labels       = np.load(train_data_labels)
observations = np.load(train_data_obs_map)
rewards      = np.load(train_data_reward)

In [7]:
labels = to_categorical(labels, num_classes=action_space)

In [8]:
labels.shape, observations.shape, rewards.shape

((553065, 6), (553065, 11, 11, 18), (553065,))

In [9]:
np.sum(labels, axis=0) / np.sum(labels)

array([0.15941164, 0.20397964, 0.190945  , 0.20009764, 0.20339382,
       0.04217226], dtype=float32)

In [10]:
class_weights = compute_class_weight('balanced', np.unique(np.argmax(labels, axis=1)), np.argmax(labels, axis=1))
class_weights

array([1.04551126, 0.81707501, 0.87285166, 0.83292671, 0.81942839,
       3.9520451 ])

In [11]:
trainer = Trainer(action_space, model_path, log_path)

In [12]:
trainer.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 11, 11, 18)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 9, 9, 256)    41728       input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 9, 9, 256)    1024        conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 9, 9, 256)    0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
flatten_1 

In [ ]:
trainer.train(observations, labels, rewards, batch_size=batch_size, 
              epochs=epochs, early_stopping=early_stopping, class_weight=[class_weights, 1])

Train on 470105 samples, validate on 82960 samples
Epoch 1/150
470105/470105 [==============================] - 15s 32us/step - loss: 2.2064 - actions_loss: 1.7125 - reward_loss: 0.4939 - actions_acc: 0.3196 - reward_acc: 0.7518 - val_loss: 2.0711 - val_actions_loss: 1.5671 - val_reward_loss: 0.5041 - val_actions_acc: 0.3421 - val_reward_acc: 0.7480
Epoch 2/150
470105/470105 [==============================] - 12s 25us/step - loss: 1.9718 - actions_loss: 1.4806 - reward_loss: 0.4912 - actions_acc: 0.3683 - reward_acc: 0.7544 - val_loss: 2.0045 - val_actions_loss: 1.5005 - val_reward_loss: 0.5040 - val_actions_acc: 0.3617 - val_reward_acc: 0.7480
Epoch 3/150
470105/470105 [==============================] - 12s 25us/step - loss: 1.9262 - actions_loss: 1.4350 - reward_loss: 0.4912 - actions_acc: 0.3878 - reward_acc: 0.7544 - val_loss: 1.9640 - val_actions_loss: 1.4600 - val_reward_loss: 0.5040 - val_actions_acc: 0.3843 - val_reward_acc: 0.7480
Epoch 4/150
470105/470105 [===================

Epoch 28/150
470105/470105 [==============================] - 13s 27us/step - loss: 1.3920 - actions_loss: 0.9565 - reward_loss: 0.4355 - actions_acc: 0.5758 - reward_acc: 0.7820 - val_loss: 1.7044 - val_actions_loss: 1.1700 - val_reward_loss: 0.5344 - val_actions_acc: 0.5192 - val_reward_acc: 0.7308
Epoch 29/150
470105/470105 [==============================] - 14s 30us/step - loss: 1.3802 - actions_loss: 0.9468 - reward_loss: 0.4334 - actions_acc: 0.5794 - reward_acc: 0.7829 - val_loss: 1.8010 - val_actions_loss: 1.2677 - val_reward_loss: 0.5332 - val_actions_acc: 0.4751 - val_reward_acc: 0.7322
Epoch 30/150
470105/470105 [==============================] - 13s 27us/step - loss: 1.3813 - actions_loss: 0.9492 - reward_loss: 0.4321 - actions_acc: 0.5779 - reward_acc: 0.7836 - val_loss: 1.6752 - val_actions_loss: 1.1389 - val_reward_loss: 0.5363 - val_actions_acc: 0.5166 - val_reward_acc: 0.7301
Epoch 31/150
470105/470105 [==============================] - 14s 30us/step - loss: 1.3717 - a

470105/470105 [==============================] - 13s 28us/step - loss: 1.2422 - actions_loss: 0.8198 - reward_loss: 0.4224 - actions_acc: 0.6297 - reward_acc: 0.7888 - val_loss: 1.5475 - val_actions_loss: 1.0083 - val_reward_loss: 0.5392 - val_actions_acc: 0.5541 - val_reward_acc: 0.7291
Epoch 56/150
470105/470105 [==============================] - 13s 29us/step - loss: 1.2412 - actions_loss: 0.8188 - reward_loss: 0.4223 - actions_acc: 0.6306 - reward_acc: 0.7888 - val_loss: 1.5245 - val_actions_loss: 0.9860 - val_reward_loss: 0.5386 - val_actions_acc: 0.5615 - val_reward_acc: 0.7295
Epoch 57/150
470105/470105 [==============================] - 13s 29us/step - loss: 1.2405 - actions_loss: 0.8182 - reward_loss: 0.4223 - actions_acc: 0.6304 - reward_acc: 0.7889 - val_loss: 1.5524 - val_actions_loss: 1.0150 - val_reward_loss: 0.5375 - val_actions_acc: 0.5520 - val_reward_acc: 0.7303
Epoch 58/150
470105/470105 [==============================] - 14s 29us/step - loss: 1.2401 - actions_loss: 

In [ ]:
print("finish")